## Import packages

In [21]:
import numpy as np
import torch
import torch.optim as optim
import os
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import scipy.stats 
from scipy.optimize import fsolve


## Load science paper 5DL1 promoter data points 

In [22]:
# Load the data from file
data=np.load("science_data_5DL1.npy")

# Extract x and y data
x_data = data[:, 0]
y_data = data[:, 1]

# Get unique x values and their corresponding indices
unique_x, unique_indices = np.unique(x_data, return_index=True)

# Extract unique y values based on unique indices
unique_y = y_data[unique_indices]

# Create unique data array with x and y values
unique_data = np.column_stack((unique_x, unique_y))

# Convert mean and variance data to torch tensors
mean_data = torch.from_numpy(unique_data[:, 0]).double()
var_data = mean_data * torch.from_numpy(unique_data[:, 1]).double()

## Differentiable Gillespie algorithm


In [23]:
# Stoichiometric matrix defining the effect of each reaction on the system
stoic_matrix = torch.tensor([[2.0, 0.0],    # Reaction 1: Promoter state goes from -1 to +1
                             [0.0, 1.0],    # Reaction 2: mRNA is produced
                             [-2.0, 0.0],   # Reaction 3: Promoter state goes from +1 to -1
                             [0.0, -1.0]])  # Reaction 4: Degradation of mRNA

def state_jump(reaction_index, stoic_matrix):
    """
    Calculate state jump vector based on the selected reaction index and stoichiometry matrix, where, 
    state vector -> state vector + state jump vector.

    Arguments:
        reaction_index: Selected reaction index
        stoic_matrix: Stoichiometry matrix

    Returns:
        State jump vector
    """ 
    return torch.sum(stoic_matrix * (torch.exp(-b_inv * (reaction_index - torch.arange(stoic_matrix.shape[0]))**2)).view(-1, 1), dim=0)

def reaction_selection(breaks, random_num):
    """
    Select reaction based on the transition points and a random number. Transition points are 
    given by the ratio of cumulative sum of rates and the total rate.

    Arguments:
        breaks: Transition points between [0,1]
        random_num: Random number in [0,1]

    Returns:
        Index of the next reaction
    """
    return torch.sum(torch.sigmoid(a_inv * (random_num - breaks)))

def gillespie_simulation(poff_values, r, g):
    """
    Perform differentiable Gillespie simulation for a 2-state promoter model.
    
    Arguments:
        poff_values: Array of probabilities for promoter to be in OFF state. poff=koff/(kon+koff)
        r: Rate of mRNA production.
        g: Rate of mRNA degradation.
        
    Returns:
        mean_final_states: Mean of the mRNA levels at the end of the simulation.
        variances: Variance of the mRNA levels at the end of the simulation.
    """
    # Initialize random seed for reproducibility
    random_seed = torch.randint(1, 10000000, (1,))
    torch.manual_seed(random_seed)

    mean_final_states = torch.empty(len(unique_data))
    variances = torch.empty(len(unique_data))
    
    for n in range(len(unique_data)):
        poff = poff_values[n].unsqueeze(0)
        
        final_states = 0.0
        final_states_squared = 0.0

        for j in range(num_simulations):
            # Initial 'levels':
            # The first component of 'levels' is the promoter state, initialized to -1
            # The second component of 'levels' is the mRNA level, initialized to 0.
            levels = torch.stack([torch.tensor(-1.0), torch.tensor(0.0)])
            current_time = 0.0

            while current_time < sim_time:
                # Calculate reaction propensities
                propensities = torch.stack([(1/poff-1.0) * torch.sigmoid(-c*levels[0]), 
                                            r * torch.sigmoid(-c*levels[0]), 
                                            torch.tensor([1.0]) * torch.sigmoid(c * levels[0]), 
                                            g * levels[1]])
                propensities = torch.relu(propensities)

                # Sum of all propensities
                total_propensity = propensities.sum()

                # Time until next reaction
                dt = -torch.log(torch.rand(1)) / total_propensity
                current_time += dt.item()

                if current_time >= sim_time:
                    break

                # Determine which reaction occurs and update the system state
                breaks = (propensities[:-1] / total_propensity).cumsum(dim=0)
                reaction_index = reaction_selection(breaks, torch.rand(1))
                levels = levels + state_jump(reaction_index, stoic_matrix)
                levels[1] = torch.relu(levels[1])  # Ensure non-negative values for the mRNA number

            # Accumulate mRNA level and its square
            final_states += levels[1]
            final_states_squared += levels[1] ** 2

        # Calculate mean and variance of mRNA levels
        mean_final_state = final_states / num_simulations
        variance = final_states_squared / num_simulations - mean_final_state ** 2
        
        mean_final_states[n] = mean_final_state
        variances[n] = variance

    return mean_final_states, variances

def loss_function(mean_final_states, variances):
    """
    Loss function that calculates the mean squared error of the simulation results against data.

    Arguments:
        mean_final_states: Mean of the mRNA levels at the end of the simulation.
        variances: Variance of the mRNA levels at the end of the simulation.
        
    Returns:
        Loss value
    """
    return torch.mean((mean_final_states - mean_data)**2 + (variances**0.5 - var_data**0.5)**2)

# Define a function to write data to a file
def write_to_file(filename, *args):
    with open(filename, 'a') as file:
        file.write(' '.join(map(str, args)) + '\n')


## Extract the learned parameter values

In [24]:
# Find the row index with minimum loss value. 
data=np.loadtxt("learning_science_5DL1.txt")
row_index = np.argmin(data[:, -1])
r=data[row_index,1]
g=data[row_index, 2]

# Path to poff_values
file_path = 'learning_science_5DL1_poff.txt'

# Initialize an empty list to store rows
poff_values = []

with open(file_path, 'r') as file:
    for line in file:
        # Remove the trailing number by splitting at the last space and taking the first part
        list_str = line.rsplit(' ', 1)[0]
        # Remove surrounding brackets
        list_str = list_str.strip('[').strip(']')
        # Split the remaining string by commas to form a list of strings
        str_values = list_str.split(',')
        # Convert strings to floats
        row = [float(value) for value in str_values]
        # Append the row to our data list
        poff_values.append(row)
  
poff_values = poff_values[row_index][:len(unique_data)]    

## Standard deviation of r


In [26]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define simulation hyperparameters
num_points = 50
num_simulations = 200
a_inv=200.0
b_inv=20.0
c=20.0
sim_time = 0.2
num_points2 = 50

# Initialize lists to store loss
loss_list = []

# Generate a range of r values around the optimal value
r_list = np.linspace(r - r * 0.8, r * 2, num_points)

# Perform simulation for each r value
for r_value in r_list:
    # Perform Gillespie simulation
    mean, var = gillespie_simulation(torch.tensor(poff_values), torch.tensor([r_value]), torch.tensor([g]))
    # Calculate loss based on simulation results
    loss = loss_function(mean, var)
    loss_list.append(loss.item())

# Polynomial fitting
degree = 6
# Fit a polynomial curve to the loss data
coeffs = np.polyfit(r_list, loss_list, degree)
poly_func = np.poly1d(coeffs)
# Calculate first and second derivatives of the polynomial curve
first_derivative = np.polyder(poly_func, 1)
second_derivative = np.polyder(poly_func, 2)

# Define evaluation points for curvature analysis
evaluation_points = np.linspace(r_list[0], r_list[-1], num_points2)

# Evaluate curvature at each point
curvatures = [second_derivative(point) for point in evaluation_points]

# Compute the average curvature
average_curvature = np.mean(curvatures)

# Compute standard deviation based on average curvature
std_r = 1 / np.sqrt(abs(average_curvature))

# Print the computed standard deviation
print(std_r)

28.98746767133203


## Standard deviation of $\gamma$

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define simulation hyperparameters
num_points = 50
num_simulation = 200
a_inv=200.0
b_inv=20.0
c=20.0
sim_time = 0.2
num_points2 = 50

# Initialize lists to store loss
loss_list = []

# Generate a range of g values around the optimal value
g_list = np.linspace(g-g*0.8, g*2, num_points)

# Perform simulation for each g value
for g_value in g_list:
    mean, var = gillespie_simulation(torch.tensor(poff_values), torch.tensor([r]), torch.tensor([g_value]))
    loss = loss_function(mean, var)
    loss_list.append(loss.item())

# Polynomial fitting
degree = 6
# Fit a polynomial curve to the loss data
coeffs = np.polyfit(g_list, loss_list, degree)
poly_func = np.poly1d(coeffs)
# Calculate first and second derivatives of the polynomial curve
first_derivative = np.polyder(poly_func, 1)
second_derivative = np.polyder(poly_func, 2)

# Define evaluation points for curvature analysis
evaluation_points = np.linspace(g_list[0], g_list[-1], num_points2)

# Evaluate curvature at each point
curvatures = [second_derivative(point) for point in evaluation_points]

# Compute the average curvature
average_curvature = np.mean(curvatures)

# Compute standard deviation based on average curvature
std_g = 1 / np.sqrt(abs(average_curvature))

# Print the computed standard deviation
print(std_g)

## Standard deviation of $r/\gamma$ using propagation of error

In [ ]:
std_r_g = (r/g) * ((std_r/r)**2 + (std_g/g)**2)**0.5
print(std_r_g)